## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv') #открываем файл
df.info() #получаем общую информацию по файлу

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Файл содержит таблицу из 12 столбцов и 21525 строк. В столбцах days_employed и total_income существуют пропущенные значения, которые необходимо обработать для получения корректных выводов. Столбцы содержат 3 типа данных, есть как целые числа, так и значения с плавающей запятой.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
for row in df['dob_years'].unique():
    mediana_for_days_employed = df.loc[df['dob_years'] == row, 'days_employed'].median()
    df.loc[(df['dob_years'] == row)&(df['days_employed'].isna()), 'days_employed'] = mediana_for_days_employed
    
    mediana_for_total_income  = df.loc[df['dob_years'] == row, 'total_income'].median()
    df.loc[(df['dob_years'] == row)&(df['total_income'].isna()), 'total_income'] = mediana_for_total_income
df.isna().sum() #демонстрирую, что пропусков нет


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

Примерно в 10% строчек нет данных по общему трудовому стажу в днях и ежемесячному доходу. Возможно произашел тех.сбой при передаче данных, т.к. их нет в одних и тех же строках и поле, в особеннности ежемесячный доход, являются важным для оформления кредита.Так как пропущенных значений весьма много, мы не можем удалить данные строки и не учитывать их при дальнейшей работе. Поэтому нам необходимо заменить значения этих строчек на иные. Замена на 0 или среднее значение по столбцу сильно исказит данные, поэтому можно заменить значения на медианные, опираясь на возраст. Так как таблица содержит большой массив данных, то опираться на возраст возможно, однако если бы данных было меньше, то лучше использовать столбец с типом занятости, введу наличия в нем меньшего кол-ва значений.

### Замена типа данных

In [3]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
# проверим на уникальность все значения в столбцах, ниже оставил только те, где есть ошибки в данных
df['children'].unique()
df['children'] = df['children'].replace(-1,1) #детей не может быть -1, здесь опечатка, значения заменили на очечидные
df[df['children'] == 20]['children'].count() #20 детей не может быть у ряда людей из данного списка, предполагаем, что это очередная опечатка и меняем значения с 20 на 2
df['children'] = df['children'].replace(20,2) 
df['dob_years'].unique()
df[df['dob_years'] == 0] #возраст клиентов в данной выборке не может быть равен 0, а значит нам необходимо заменить эти значения, например на средние. Однако мы этого дедать не будем, так как на ответы на итоговые вопросы это не повлияет
df['gender'].unique()
df[df['gender'] == 'XNA'] #ошибка в данных, замена не возможна, на результат анализа не повлияет
df['education'].unique() 
df['education'] = df['education'].str.lower() #разный регистр, приводим все к нижнему для качественного анализа в дальнейшем


### Вывод

Для удобства восприятия информации переведем трудовой стаж и ежемесячный доход в формат int, то есть уберем значения после запятой. Необходимо передать поставщику данных, чтобы проверил, почему возраст некоторых клиентов может получиться 0 (например убрать тех.возможность сохранения карточки клиента, если в ячейке с возрастом цифра меньше 18) и кол-во детей 20 ( возможно выводить окно на экран, уточняющее достоверность данных при кол-ве детей от 3х)


### Обработка дубликатов

In [4]:
df.duplicated().sum()
df.drop_duplicates().reset_index(drop = True)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,-4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21450,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21451,1,-2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21452,3,-3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


### Вывод

Внутри любого столбца могут быть повторяющиеся значения, а значит использование метода value_counts() невозможно. Поэтому я применил метод df.duplicated().sum(), чтобы понять кол-во строчек - дибликатов и далее удалив их с заменой индексов. Скорее всего наличие 2х абсолютно одинаковых строчек связано с тех.ошибками, так как мало вероятно, что 1 человек может взять 2 кредита имея один и тот же трудовой стаж в днях и прочие параметры. Хотя конечно это и возможно. 

### Лемматизация

In [5]:
df['purpose'].unique()
from pymystem3 import Mystem #вызываем библиотеку pymastem для проведения лемматизации
m = Mystem()

def value_purpose(purpose):
    lemmas = ''.join(m.lemmatize(purpose))
    return lemmas

df['value_purpose'] = df['purpose'].apply(value_purpose) #добавляем новый столбец с леммами из столбца с целью кредита
df['value_purpose'].sum() 
from collections import Counter
Counter(df['value_purpose']) #выясняем сколько раз встречается каждая лемма 
try:
    def lemma_summary(value_purpose):
        if 'жилье' in value_purpose:
            return 'недвижимость'
        if 'автомобиль' in value_purpose:
            return 'автомобиль'
        if 'образование' in value_purpose:
            return 'образование'
        if 'свадьба' in value_purpose:
            return 'свадьба'
        if 'недвижимость' in value_purpose:
            return 'недвижимость'
    df['lemma_summary'] = df['value_purpose'].apply(lemma_summary)
except:
    'Проверь пропуски и типы данных'
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,value_purpose,lemma_summary
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье\n,недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль\n,автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье\n,недвижимость
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование\n,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба\n,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,операция с жилье\n,недвижимость
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,сделка с автомобиль\n,автомобиль
21522,1,-2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость\n,недвижимость
21523,3,-3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,на покупка свой автомобиль\n,автомобиль


### Вывод

Всего в предоставленном списке выделено 4 причины для того, чтобы взять кредит: недвижимость, автомобиль, свадьба и образование. Банка можно также предложить создать категории целей кредита по основным + оставить категорию другая цель, где и можно прописывать комментарий. Это позволит избежать лемматизации каждый раз при работе с данными.

### Категоризация данных

In [6]:
children_table = df[['children','debt',]] #словарь для ответа на 1ый вопрос про детей

children_table = df.groupby('children').agg({'debt': ['count','sum']})
children_table['conversion'] = children_table['debt']['sum'] / children_table['debt']['count'] * 100
children_table["conversion"] = children_table["conversion"].astype('int')
#сгруппировали данные по одному столбцу и провели подсчет кол-ва и суммы по другому, далее высчитали конверсию, добавили новый столбец и поменяли в нем формат

family_table = df[['family_status', 'family_status_id','debt',]] #словарь для ответа на 2ой вопрос про семейное положение

family_table = df.groupby('family_status_id').agg({'debt': ['count','sum']})
family_table['conversion'] = family_table['debt']['sum'] / family_table['debt']['count'] * 100
family_table["conversion"] = family_table["conversion"].astype('int')

def incom(total_income):
    if total_income < 70000:
        return 'з.п. до 70 000р в месяц'
    elif total_income < 120000:
        return 'з.п. от 70 до 120 000р в месяц'
    elif total_income < 170000:
        return 'з.п. от 120 до 170 тысяч в месяц'
    else:
        return 'з.п. более 170 тысяч в месяц'
df['incom'] = df['total_income'].apply(incom)   #создадим новый столбец в таблице с категориями з.п. клиентов банка.
#Избежим названий "бедный" и 'богатый', сделаем кетегории максимально близкими по доли в кол-ве. При работе с заказчиком эти категории лучше уточнить у него или операться на данный гор.органов
income_table = df[['total_income','debt','incom']] #словарь для ответа на 3ий вопрос про доход 

income_table = df.groupby('incom').agg({'debt': ['count','sum']})
income_table['conversion'] = income_table['debt']['sum'] / income_table['debt']['count'] * 100
income_table["conversion"] = income_table["conversion"].astype('int')
 

purpose_table = df[['purpose','debt',]] #словарь для ответа на 4ый вопрос про цель кредита

purpose_table = df.groupby('lemma_summary').agg({'debt': ['count','sum']})
purpose_table['conversion'] = purpose_table['debt']['sum'] / purpose_table['debt']['count'] * 100
purpose_table["conversion"] = purpose_table["conversion"].astype('int')

full_table = df.pivot_table(index = ['lemma_summary', 'children', 'family_status_id', 'incom'], values = 'debt', aggfunc = ['count','sum','mean'])
full_table = full_table.reset_index() # объединяем все таблицы выше в одну
full_table.columns = ['lemma_summary', 'children', 'family_status_id', 'incom','count','sum','conversion'] #переименовываем столбцы во избежание двойных названий
full_table['conversion'] = full_table['conversion'] * 100
full_table['conversion'] = full_table['conversion'].astype('int') #меняем формат в столбце с конверсией для лучшего восприятия
full_table_full = full_table.sort_values(by = 'conversion', ascending = False) 
finaliti = full_table_full[full_table_full['count'] > 50] #сортируем таблицу по столбцу с конверсий и оставляем в ней только те строки, где кол-во клиентов является репрезентативной выборкой, тое есть хотя бы больще 50ч
finaliti
#finaliti.head(5)
#finaliti.tail(5) таблицы нужны для финального вывода

,lemma_summary,children,family_status_id,incom,count,sum,conversion
168,образование,0,1,з.п. от 70 до 120 000р в месяц,69,14,20
102,недвижимость,1,1,з.п. от 120 до 170 тысяч в месяц,89,15,16
7,автомобиль,0,1,з.п. от 70 до 120 000р в месяц,73,11,15
204,образование,2,0,з.п. от 70 до 120 000р в месяц,90,14,15
112,недвижимость,1,4,з.п. более 170 тысяч в месяц,91,13,14
...,...,...,...,...,...,...,...
84,недвижимость,0,2,з.п. более 170 тысяч в месяц,119,6,5
242,свадьба,2,1,з.п. более 170 тысяч в месяц,74,3,4
171,образование,0,2,з.п. от 120 до 170 тысяч в месяц,63,2,3
87,недвижимость,0,2,з.п. от 70 до 120 000р в месяц,145,3,2


### Вывод

Итого мы получили таблицу из 81 строки, сожержащую все необходимые зависимости между показателями по заданным вопросам. Данный файл можно использовать для модели кредитного скоринга.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
children_table 

debt       conversion
          count   sum           
children                        
0         14149  1063          7
1          4865   445          9
2          2131   202          9
3           330    27          8
4            41     4          9
5             9     0          0

### Вывод

Клиенты банка с 1-2 детьми в среднем на 2% реже имеют задолженность по возрату кредита в сравнении с клиентами, у которых нет детей. Предоставленных данных с детьми от 3х недостаточно для корректного анализа данной категории. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
family_table

debt      conversion
                  count  sum           
family_status_id                       
0                 12380  931          7
1                  4177  388          9
2                   960   63          6
3                  1195   85          7
4                  2813  274          9

### Вывод

Лучшими клиентами с точки зрения отсутсвия задолженности по кредиту являются клиенты в гражданском браке или незамужние/неженатые (по 9%), далее идут клиенты в разводе и женатые/замужние (по 7%), далее идут вдовцы/вдовы (6%)

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
income_table

debt      conversion
                                 count  sum           
incom                                                 
з.п. более 170 тысяч в месяц      7216  547          7
з.п. до 70 000р в месяц           1474  101          6
з.п. от 120 до 170 тысяч в месяц  7402  638          8
з.п. от 70 до 120 000р в месяц    5433  455          8

### Вывод

Лучшими клиентами с точки зрения отсутсвия задолженности по кредиту являются клиенты с зарплатой ежемесячной в диапазоне от 70 до 170 тысяч рублей ( по 8%), клиенты же с зарплатой свыше 170 тысяч на 1% чаще имеют задолженность, а клиенты с зарлатой до 70 тысяч на 2%. 

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
purpose_table

debt      conversion
               count  sum           
lemma_summary                       
автомобиль      4315  403          9
недвижимость   10840  782          7
образование     4022  370          9
свадьба         2348  186          7

### Вывод

Клиенты банка, берущие кредит на автомобиль и образование 9% случаев не имеют задолженность по кредиту, а клиенты берущие кредит на нанедвижимость и свадьбу в 7%

### Шаг 4. Общий вывод

Лучше всего возвращают кредиты:  
    1) Клиенты в гражданском браке, не имеющие детей, зарабатывающие от 70 до 120 тысяч и взявшие кредит на образование  
    2) Клиенты в гражданском браке с 1 ребенком, зарабатывающие от 120 до 170 тысяч в месяц и взявшие кредит на недвижимость  
    3) Клиенты в гражданском браке без детей, зарабатывающие от 70 до 120 тысяч и взявшие кредит на автомобиль  
    4) Клиенты в браке с 2мя детьми, зарабатывающие от 70 до 120 тысяч и взявшие кредит на образование  
    5) Клиенты неженатые с 1 ребенком, зарабатывающие  170 тысяч в месяц и взявшие кредит на недвидимость

Хуже всего возвращают кредиты:  
    1) Вдовцы/вдовы с 2мя детьми с зарлатой более 170 тысяч рублей и взявшие кредит на недвижимость  
    2) Клиенты в гражданском браке с 2мя детьми с зарлатой более 170 тысяч рублей и взявшие кредит на свадьбу  
    3) Вдовцы/вдовы без детей с зарплатой от 120 до 170 тысяч и взявшие кредит на образование  
    4) Вдовцы/вдовы без детей с зарплатой от 70 до 120 тысяч и взявшие кредит на недвижимость  
    5) Клиенты в разводе без детей с зарлатой более 170 тысяч рублей и взявшие кредит на автомобиль  